In [2]:
# Import necessary libraries
import asyncio
import httpx
import json
import time
from typing import Dict, Any
import pandas as pd

# FastAPI testing client
from httpx import AsyncClient

print("All libraries imported successfully!")
print("Available modules:")
print("- asyncio: For async operations")
print("- httpx: For HTTP client requests")
print("- json: For JSON data handling")
print("- time: For performance measurements")
print("- pandas: For data analysis")

All libraries imported successfully!
Available modules:
- asyncio: For async operations
- httpx: For HTTP client requests
- json: For JSON data handling
- time: For performance measurements
- pandas: For data analysis


In [3]:
# API Configuration
BASE_URL = "http://localhost:8011"  # Adjust port if needed
API_ENDPOINTS = {
    "health": f"{BASE_URL}/",
    "users": f"{BASE_URL}/users",
    "messages": f"{BASE_URL}/messages",
    "chat": f"{BASE_URL}/chat"
}

# Test configuration
TIMEOUT_SECONDS = 30
RETRY_ATTEMPTS = 3

print("API Configuration:")
for name, url in API_ENDPOINTS.items():
    print(f"- {name}: {url}")
print(f"- Timeout: {TIMEOUT_SECONDS} seconds")
print(f"- Retry attempts: {RETRY_ATTEMPTS}")

API Configuration:
- health: http://localhost:8011/
- users: http://localhost:8011/users
- messages: http://localhost:8011/messages
- chat: http://localhost:8011/chat
- Timeout: 30 seconds
- Retry attempts: 3


In [ ]:
async def check_server_health():
    """Check if the FastAPI server is running"""
    try:
        async with httpx.AsyncClient(timeout=TIMEOUT_SECONDS) as client:
            response = await client.get(API_ENDPOINTS["health"])
            if response.status_code == 200:
                data = response.json()
                print("Server Health Check: PASSED")
                print(f"Response: {json.dumps(data, indent=2)}")
                return True
            else:
                print(f"Server Health Check: FAILED (Status: {response.status_code})")
                return False
    except Exception as e:
        print(f"Server Health Check: FAILED (Error: {str(e)})")
        print("Make sure the FastAPI server is running with: python main.py")
        return False

health_status = await check_server_health()

Server Health Check: PASSED
Response: {
  "message": "Welcome to Bot API",
  "status": "running"
}


In [40]:
async def create_test_user():
    """Create a test user and return the user ID"""
    try:
        async with httpx.AsyncClient(timeout=TIMEOUT_SECONDS) as client:
            response = await client.post(API_ENDPOINTS["users"])
            if response.status_code == 201:
                data = response.json()
                user_id = data.get('id')
                print(f"Created test user with ID: {user_id}")
                return user_id
            else:
                print(f"Failed to create user (Status: {response.status_code})")
                return None
    except Exception as e:
        print(f"Error creating user: {str(e)}")
        return None

test_user_id = await create_test_user()

Created test user with ID: ad8b792e-98ab-4940-93b9-9177d0f68e24


In [37]:
async def test_basic_chat(message: str, user_id: str = None):
    """Test basic chat functionality"""
    # Use test_user_id if no user_id provided
    if user_id is None:
        user_id = test_user_id
    
    print(f"Testing chat with message: '{message}'")
    print(f"User ID: {user_id}")
    print("-" * 50)
    
    try:
        async with httpx.AsyncClient(timeout=TIMEOUT_SECONDS) as client:
            payload = {
                "message": message,
                "user_id": user_id
            }
            
            start_time = time.time()
            response = await client.post(API_ENDPOINTS["chat"], json=payload)
            end_time = time.time()
            
            response_time = end_time - start_time
            
            print(f"Status Code: {response.status_code}")
            print(f"Response Time: {response_time:.2f} seconds")
            
            if response.status_code == 200:
                data = response.json()
                print("SUCCESS!")
                print(f"AI Response: {data.get('bot_response', 'No response found')}")
                return data
            else:
                print("FAILED!")
                print(f"Error: {response.text}")
                return None
                
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return None

# Test with a simple greeting
test_result = await test_basic_chat("Hello! How are you today?")

Testing chat with message: 'Hello! How are you today?'
User ID: 03a81586-2770-4add-874c-886e7b1a6ea4
--------------------------------------------------
Status Code: 200
Response Time: 1.58 seconds
SUCCESS!
AI Response: Hello! I'm functioning perfectly, thank you. I'm here and ready to help you with any questions or problems you might have. How can I assist you today?
Status Code: 200
Response Time: 1.58 seconds
SUCCESS!
AI Response: Hello! I'm functioning perfectly, thank you. I'm here and ready to help you with any questions or problems you might have. How can I assist you today?


## 5. Error Handling Tests

Test how the system handles invalid requests and edge cases.

In [26]:
async def test_error_scenarios():
    """Test various error scenarios"""
    
    test_cases = [
        {
            "name": "Empty message",
            "payload": {"message": "", "user_id": test_user_id},
            "expected_status": 422
        },
        {
            "name": "Missing message field",
            "payload": {"user_id": test_user_id},
            "expected_status": 422
        },
        {
            "name": "Missing user_id field",
            "payload": {"message": "Hello"},
            "expected_status": 422
        },
        {
            "name": "Very long message",
            "payload": {"message": "A" * 5000, "user_id": test_user_id},
            "expected_status": 200
        }
    ]
    
    print("Running error handling tests...")
    print("=" * 60)
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"Test {i}: {test_case['name']}")
        
        try:
            async with httpx.AsyncClient(timeout=TIMEOUT_SECONDS) as client:
                response = await client.post(API_ENDPOINTS["chat"], json=test_case["payload"])
                
                print(f"Status Code: {response.status_code}")
                
                if response.status_code == test_case["expected_status"]:
                    print("RESULT: PASSED")
                else:
                    print(f"RESULT: FAILED (Expected {test_case['expected_status']}, got {response.status_code})")
                
                if response.status_code != 200:
                    print(f"Error Response: {response.text}")
                    
        except Exception as e:
            print(f"ERROR: {str(e)}")
            
        print("-" * 40)

# Run error tests
await test_error_scenarios()

Running error handling tests...
Test 1: Empty message


Status Code: 200
RESULT: FAILED (Expected 422, got 200)
----------------------------------------
Test 2: Missing message field
Status Code: 422
RESULT: PASSED
Error Response: {"detail":[{"type":"missing","loc":["body","message"],"msg":"Field required","input":{"user_id":"c8d728ea-d842-452d-a95d-144ec9885a18"}}]}
----------------------------------------
Test 3: Missing user_id field
Status Code: 422
RESULT: PASSED
Error Response: {"detail":[{"type":"missing","loc":["body","message"],"msg":"Field required","input":{"user_id":"c8d728ea-d842-452d-a95d-144ec9885a18"}}]}
----------------------------------------
Test 3: Missing user_id field
Status Code: 422
RESULT: PASSED
Error Response: {"detail":[{"type":"missing","loc":["body","user_id"],"msg":"Field required","input":{"message":"Hello"}}]}
----------------------------------------
Test 4: Very long message
Status Code: 422
RESULT: PASSED
Error Response: {"detail":[{"type":"missing","loc":["body","user_id"],"msg":"Field required","input":{

## 6. Different Message Types Testing

Test the AI with various types of questions and conversation styles.

In [27]:
async def test_message_varieties():
    """Test different types of messages and conversation styles"""
    
    test_messages = [
        "Hello! What's your name?",
        "Can you help me with programming?",
        "What's the weather like today?",
        "Tell me a joke",
        "What is the capital of France?",
        "How do I learn Python programming?",
        "What's 2 + 2?",
        "Can you write a poem about cats?",
        "Explain quantum physics in simple terms"
    ]
    
    print("Testing different message types...")
    print("=" * 60)
    
    results = []
    
    for i, message in enumerate(test_messages, 1):
        print(f"Test {i}/{len(test_messages)}: {message}")
        print("-" * 40)
        
        start_time = time.time()
        result = await test_basic_chat(message, test_user_id)
        end_time = time.time()
        
        if result:
            response_time = end_time - start_time
            results.append({
                "message": message,
                "response": result.get("bot_response", ""),
                "response_time": response_time,
                "success": True
            })
        else:
            results.append({
                "message": message,
                "response": "",
                "response_time": 0,
                "success": False
            })
        
        print("\\n" + "=" * 60)
    
    # Summary
    successful_tests = sum(1 for r in results if r["success"])
    print(f"\\nSUMMARY:")
    print(f"Total tests: {len(test_messages)}")
    print(f"Successful: {successful_tests}")
    print(f"Failed: {len(test_messages) - successful_tests}")
    
    if successful_tests > 0:
        avg_response_time = sum(r["response_time"] for r in results if r["success"]) / successful_tests
        print(f"Average response time: {avg_response_time:.2f} seconds")
    
    return results

# Run message variety tests
message_test_results = await test_message_varieties()

Testing different message types...
Test 1/9: Hello! What's your name?
----------------------------------------
Testing chat with message: 'Hello! What's your name?'
User ID: c8d728ea-d842-452d-a95d-144ec9885a18
--------------------------------------------------
Status Code: 200
Response Time: 1.35 seconds
SUCCESS!
AI Response: Hello! You can call me Assistant. I'm here to help you with any questions or information you need. How can I assist you today?
\n============================================================
Test 2/9: Can you help me with programming?
----------------------------------------
Testing chat with message: 'Can you help me with programming?'
User ID: c8d728ea-d842-452d-a95d-144ec9885a18
--------------------------------------------------
Status Code: 200
Response Time: 1.35 seconds
SUCCESS!
AI Response: Hello! You can call me Assistant. I'm here to help you with any questions or information you need. How can I assist you today?
\n========================================

## 7. Performance Testing

Test response times and concurrent request handling.

In [30]:
async def test_performance():
    """Test performance with multiple concurrent requests"""
    
    print("Performance Testing...")
    print("=" * 50)
    
    # Test single request performance
    print("1. Single Request Performance:")
    start_time = time.time()
    result = await test_basic_chat("What is artificial intelligence?", test_user_id)
    end_time = time.time()
    single_request_time = end_time - start_time
    print(f"Single request time: {single_request_time:.2f} seconds\\n")
    
    # Test concurrent requests
    print("2. Concurrent Requests Test:")
    concurrent_requests = 5
    test_message = "Hello, this is a concurrent test!"
    
    async def single_concurrent_request(user_num):
        """Single request for concurrent testing"""
        try:
            async with httpx.AsyncClient(timeout=TIMEOUT_SECONDS) as client:
                payload = {
                    "message": f"{test_message} (User {user_num})",
                    "user_id": test_user_id
                }
                start = time.time()
                response = await client.post(API_ENDPOINTS["chat"], json=payload)
                end = time.time()
                return {
                    "user_num": user_num,
                    "success": response.status_code == 200,
                    "response_time": end - start,
                    "status_code": response.status_code
                }
        except Exception as e:
            return {
                "user_num": user_num,
                "success": False,
                "response_time": 0,
                "error": str(e)
            }
    
    # Run concurrent requests
    print(f"Sending {concurrent_requests} concurrent requests...")
    concurrent_start = time.time()
    
    tasks = [single_concurrent_request(i) for i in range(1, concurrent_requests + 1)]
    concurrent_results = await asyncio.gather(*tasks)
    
    concurrent_end = time.time()
    total_concurrent_time = concurrent_end - concurrent_start
    
    # Analyze results
    successful_concurrent = sum(1 for r in concurrent_results if r["success"])
    failed_concurrent = concurrent_requests - successful_concurrent
    
    if successful_concurrent > 0:
        avg_concurrent_time = sum(r["response_time"] for r in concurrent_results if r["success"]) / successful_concurrent
        max_concurrent_time = max(r["response_time"] for r in concurrent_results if r["success"])
        min_concurrent_time = min(r["response_time"] for r in concurrent_results if r["success"])
    else:
        avg_concurrent_time = max_concurrent_time = min_concurrent_time = 0
    
    print(f"Total time for {concurrent_requests} concurrent requests: {total_concurrent_time:.2f} seconds")
    print(f"Successful requests: {successful_concurrent}")
    print(f"Failed requests: {failed_concurrent}")
    print(f"Average response time: {avg_concurrent_time:.2f} seconds")
    print(f"Min response time: {min_concurrent_time:.2f} seconds")
    print(f"Max response time: {max_concurrent_time:.2f} seconds")
    
    # Performance summary
    print("\\n" + "=" * 50)
    print("PERFORMANCE SUMMARY:")
    print(f"Single request: {single_request_time:.2f}s")
    print(f"Concurrent avg: {avg_concurrent_time:.2f}s")
    print(f"Success rate: {(successful_concurrent/concurrent_requests)*100:.1f}%")
    
    return {
        "single_request_time": single_request_time,
        "concurrent_results": concurrent_results,
        "total_concurrent_time": total_concurrent_time,
        "success_rate": (successful_concurrent/concurrent_requests)*100
    }

# Run performance tests
performance_results = await test_performance()

Performance Testing...
1. Single Request Performance:
Testing chat with message: 'What is artificial intelligence?'
User ID: c8d728ea-d842-452d-a95d-144ec9885a18
--------------------------------------------------
Status Code: 200
Response Time: 4.46 seconds
SUCCESS!
AI Response: Hello! Artificial Intelligence (AI) is a branch of computer science that aims to create machines capable of performing tasks that typically require human intelligence. These tasks include:

1. **Learning**: Acquiring information and rules for using the information.
2. **Reasoning**: Using the rules to reach approximate or definite conclusions.
3. **Problem-solving**: Finding solutions to complex problems.
4. **Perception**: Using sensory information (like vision, touch, and sound) to understand the environment.
5. **Language understanding and generation**: Communicating with humans in their own language.

AI can be categorized into two main types:

1. **Narrow AI (Weak AI)**: Designed to perform a narrow task (

## 8. Complete Test Suite

Run all tests in sequence and generate a comprehensive report.

In [31]:
async def run_complete_test_suite():
    """Run all tests and generate a comprehensive report"""
    
    print("FASTAPI BOT BACKEND - COMPLETE TEST SUITE")
    print("=" * 60)
    print(f"Test started at: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 60)
    
    test_results = {
        "start_time": time.time(),
        "health_check": False,
        "basic_chat": False,
        "error_handling": False,
        "message_varieties": False,
        "performance": False,
        "overall_success": False
    }
    
    try:
        # 1. Health Check
        print("\\n1. HEALTH CHECK")
        print("-" * 30)
        health_status = await check_server_health()
        test_results["health_check"] = health_status
        
        if not health_status:
            print("Server is not running. Cannot proceed with tests.")
            return test_results
        
        # 2. Basic Chat Test
        print("\\n2. BASIC CHAT TEST")
        print("-" * 30)
        basic_result = await test_basic_chat("Hello! This is a test message.")
        test_results["basic_chat"] = basic_result is not None
        
        # 3. Error Handling
        print("\\n3. ERROR HANDLING TESTS")
        print("-" * 30)
        await test_error_scenarios()
        test_results["error_handling"] = True
        
        # 4. Message Varieties
        print("\\n4. MESSAGE VARIETY TESTS")
        print("-" * 30)
        variety_results = await test_message_varieties()
        successful_varieties = sum(1 for r in variety_results if r["success"])
        test_results["message_varieties"] = successful_varieties > len(variety_results) * 0.7  # 70% success rate
        
        # 5. Performance Tests
        print("\\n5. PERFORMANCE TESTS")
        print("-" * 30)
        perf_results = await test_performance()
        test_results["performance"] = perf_results["success_rate"] > 80  # 80% success rate
        
    except Exception as e:
        print(f"Test suite error: {str(e)}")
    
    # Calculate overall results
    test_results["end_time"] = time.time()
    test_results["total_time"] = test_results["end_time"] - test_results["start_time"]
    
    passed_tests = sum(1 for key, value in test_results.items() 
                      if key not in ["start_time", "end_time", "total_time", "overall_success"] and value)
    total_tests = 5  # health, basic, error, varieties, performance
    
    test_results["overall_success"] = passed_tests == total_tests
    
    # Generate final report
    print("\\n" + "=" * 60)
    print("FINAL TEST REPORT")
    print("=" * 60)
    print(f"Total execution time: {test_results['total_time']:.2f} seconds")
    print(f"Tests passed: {passed_tests}/{total_tests}")
    print()
    print("Test Results:")
    print(f"  Health Check: {'PASS' if test_results['health_check'] else 'FAIL'}")
    print(f"  Basic Chat: {'PASS' if test_results['basic_chat'] else 'FAIL'}")
    print(f"  Error Handling: {'PASS' if test_results['error_handling'] else 'FAIL'}")
    print(f"  Message Varieties: {'PASS' if test_results['message_varieties'] else 'FAIL'}")
    print(f"  Performance: {'PASS' if test_results['performance'] else 'FAIL'}")
    print()
    
    if test_results["overall_success"]:
        print("ALL TESTS PASSED! Your Mistral AI chatbot is working perfectly!")
    else:
        print("Some tests failed. Check the detailed output above for issues.")
    
    print("=" * 60)
    return test_results

# Uncomment the line below to run the complete test suite
complete_results = await run_complete_test_suite()

print("\\nTest notebook is ready!")
print("\\nTo start testing:")
print("1. Make sure your FastAPI server is running (python main.py)")
print("2. Run the cells above one by one to test specific features")
print("3. Or uncomment and run the complete test suite cell above")
print("\\nEach section tests different aspects of your Mistral AI chatbot!")

FASTAPI BOT BACKEND - COMPLETE TEST SUITE
Test started at: 2025-07-28 17:17:21
\n1. HEALTH CHECK
------------------------------
Server Health Check: PASSED
Response: {
  "message": "Welcome to Bot API",
  "status": "running"
}
\n2. BASIC CHAT TEST
------------------------------
Testing chat with message: 'Hello! This is a test message.'
User ID: c8d728ea-d842-452d-a95d-144ec9885a18
--------------------------------------------------
Server Health Check: PASSED
Response: {
  "message": "Welcome to Bot API",
  "status": "running"
}
\n2. BASIC CHAT TEST
------------------------------
Testing chat with message: 'Hello! This is a test message.'
User ID: c8d728ea-d842-452d-a95d-144ec9885a18
--------------------------------------------------
Status Code: 200
Response Time: 1.41 seconds
SUCCESS!
AI Response: Hello! Nice to meet you. This is a test message too. How can I assist you today? Let's keep it friendly and helpful, just like you asked. 😊
\n3. ERROR HANDLING TESTS
-----------------------

In [7]:
# Test the original simple chat system
print("🔄 TESTING ORIGINAL SIMPLE CHAT SYSTEM")
print("=" * 50)

async with httpx.AsyncClient(timeout=15) as client:
    try:
        # Create a user first
        user_response = await client.post(f"{BASE_URL}/users")
        if user_response.status_code == 201:
            user_id = user_response.json()["id"]
            print(f"✅ User created: {user_id}")
            
            # Test simple chat endpoint
            chat_payload = {
                "user_id": user_id,
                "message": "Hello! This is a test of the original chat system."
            }
            
            response = await client.post(f"{BASE_URL}/chat", json=chat_payload)
            print(f"Chat Status: {response.status_code}")
            
            if response.status_code == 200:
                data = response.json()
                print("✅ SUCCESS! Original chat system working:")
                print(f"   💬 User Message: {data['user_message']}")
                print(f"   🤖 Bot Response: {data['bot_response'][:100]}...")
                print(f"   🆔 Message ID: {data['message_id']}")
                print(f"   ⏰ Timestamp: {data['timestamp']}")
                
                # Test another message
                chat_payload2 = {
                    "user_id": user_id,
                    "message": "Can you help me with something?"
                }
                
                response2 = await client.post(f"{BASE_URL}/chat", json=chat_payload2)
                if response2.status_code == 200:
                    data2 = response2.json()
                    print("\\n✅ Second message also successful:")
                    print(f"   🤖 Bot Response: {data2['bot_response'][:100]}...")
                
            else:
                print(f"❌ Chat failed: {response.text}")
        else:
            print(f"❌ User creation failed: {user_response.text}")
            
    except Exception as e:
        print(f"❌ Error: {e}")

print("\\n" + "=" * 50)
print("✅ ORIGINAL SIMPLE CHAT SYSTEM VERIFIED!")
print("🎯 Available endpoints:")
print("   POST /users - Create a user")
print("   POST /chat - Send a message and get AI response")
print("   GET /users/{user_id}/messages - Get user's messages")
print("=" * 50)

🔄 TESTING ORIGINAL SIMPLE CHAT SYSTEM
✅ User created: a0d5a84d-6aef-470f-a0b2-f3f7d4635be1
Chat Status: 200
✅ SUCCESS! Original chat system working:
   💬 User Message: Hello! This is a test of the original chat system.
   🤖 Bot Response: Hello! Nice to meet you. I'm here and ready to assist you. Let's test the system. How about I start ...
   🆔 Message ID: c6b6b3aa-c35f-4318-937e-9e5119afad4f
   ⏰ Timestamp: 2025-07-29T10:37:19.855538
\n✅ Second message also successful:
   🤖 Bot Response: Of course! I'm here to help. What do you need assistance with?...
\n==================================================
✅ ORIGINAL SIMPLE CHAT SYSTEM VERIFIED!
🎯 Available endpoints:
   POST /users - Create a user
   POST /chat - Send a message and get AI response
   GET /users/{user_id}/messages - Get user's messages


In [13]:
# Test the new page-based chat functionality with query parameters
print("🆕 TESTING NEW PAGE-BASED CHAT FUNCTIONALITY WITH QUERY PARAMETERS")
print("=" * 60)

async with httpx.AsyncClient(timeout=15) as client:
    try:
        # Create a user first
        user_response = await client.post(f"{BASE_URL}/users")
        if user_response.status_code == 201:
            user_id = user_response.json()["id"]
            print(f"✅ User created: {user_id}")
            
            # Test 1: Create new page (no chat_id query parameter)
            print("\\n1️⃣ CREATING NEW PAGE (no chat_id):")
            chat_payload_1 = {
                "user_id": user_id,
                "message": "Hello! This should create a new page."
            }
            
            response1 = await client.post(f"{BASE_URL}/chat", json=chat_payload_1)
            print(f"Status: {response1.status_code}")
            
            if response1.status_code == 200:
                data1 = response1.json()
                chat_id = data1['chat_id']  # Get the created chat_id
                print("✅ SUCCESS! New page created:")
                print(f"   📄 Page ID: {chat_id}")
                print(f"   💬 User Message: {data1['user_message']}")
                print(f"   🤖 Bot Response: {data1['bot_response'][:50]}...")
                
                # Test 2: Send more messages to the same page using query parameter
                print("\\n2️⃣ SENDING MESSAGES TO EXISTING PAGE (using query parameter):")
                for i in range(2):
                    chat_payload_2 = {
                        "user_id": user_id,
                        "message": f"This is message {i+2} in the same page."
                    }
                    
                    # Use query parameter for chat_id
                    response2 = await client.post(f"{BASE_URL}/chat?chat_id={chat_id}", json=chat_payload_2)
                    if response2.status_code == 200:
                        data2 = response2.json()
                        print(f"   ✅ Message {i+2}: {data2['bot_response'][:40]}...")
                        print(f"      Same Page ID: {data2['chat_id'] == chat_id}")
                
                # Test 3: Get all messages for this page
                print("\\n3️⃣ RETRIEVING PAGE MESSAGES:")
                messages_response = await client.get(f"{BASE_URL}/chat/{chat_id}/messages")
                if messages_response.status_code == 200:
                    messages = messages_response.json()
                    print(f"✅ Retrieved {len(messages)} messages from page:")
                    for i, msg in enumerate(messages, 1):
                        print(f"   Message {i}: {msg['user_message'][:30]}...")
                        print(f"     AI: {msg['assistant_message'][:30]}...")
                
                # Test 4: Create another page (no query parameter)
                print("\\n4️⃣ CREATING SECOND PAGE:")
                chat_payload_3 = {
                    "user_id": user_id,
                    "message": "This should create a different page."
                }
                
                response3 = await client.post(f"{BASE_URL}/chat", json=chat_payload_3)
                if response3.status_code == 200:
                    data3 = response3.json()
                    new_chat_id = data3['chat_id']
                    print(f"✅ Second page created: {new_chat_id}")
                    print(f"   Different from first page: {new_chat_id != chat_id}")
            
            else:
                print(f"❌ Chat failed: {response1.text}")
        else:
            print(f"❌ User creation failed: {user_response.text}")
            
    except Exception as e:
        print(f"❌ Error: {e}")

print("\\n" + "=" * 60)
print("🎯 NEW PAGE-BASED CHAT SYSTEM EXPLANATION:")
print("📝 How it works:")
print("   1. Send message WITHOUT chat_id query param → Creates new page")
print("   2. Send message WITH chat_id query param → Adds to existing page")  
print("   3. All messages in same page are grouped together")
print("   4. Use GET /chat/{chat_id}/messages to see page history")
print("\\n✅ PERFECT! Your page-based chat system is working!")
print("=" * 60)

🆕 TESTING NEW PAGE-BASED CHAT FUNCTIONALITY WITH QUERY PARAMETERS
✅ User created: b36f90e5-1f37-4a0c-9e80-0d1a5bd42a8a
\n1️⃣ CREATING NEW PAGE (no chat_id):
✅ User created: b36f90e5-1f37-4a0c-9e80-0d1a5bd42a8a
\n1️⃣ CREATING NEW PAGE (no chat_id):
Status: 200
✅ SUCCESS! New page created:
   📄 Page ID: bd7a05ac-cac3-405d-a378-cd6980d5f903
   💬 User Message: Hello! This should create a new page.
   🤖 Bot Response: Hello! I'm here to help. I don't have the ability ...
\n2️⃣ SENDING MESSAGES TO EXISTING PAGE (using query parameter):
Status: 200
✅ SUCCESS! New page created:
   📄 Page ID: bd7a05ac-cac3-405d-a378-cd6980d5f903
   💬 User Message: Hello! This should create a new page.
   🤖 Bot Response: Hello! I'm here to help. I don't have the ability ...
\n2️⃣ SENDING MESSAGES TO EXISTING PAGE (using query parameter):
   ✅ Message 2: Got it! You're continuing on the same pa...
      Same Page ID: True
   ✅ Message 2: Got it! You're continuing on the same pa...
      Same Page ID: True
   ✅ Mes

In [14]:
# Test edge cases and conversation context features
print("🛡️ TESTING EDGE CASES & CONVERSATION CONTEXT")
print("=" * 60)

async with httpx.AsyncClient(timeout=20) as client:
    try:
        # Create a user first
        user_response = await client.post(f"{BASE_URL}/users")
        if user_response.status_code == 201:
            user_id = user_response.json()["id"]
            print(f"✅ User created: {user_id}")
            
            # Test 1: Invalid chat_id (edge case)
            print("\\n1️⃣ TESTING INVALID CHAT_ID (Edge Case):")
            invalid_payload = {
                "user_id": user_id,
                "message": "This should fail with invalid chat_id"
            }
            
            # Use query parameter for invalid chat_id
            response_invalid = await client.post(f"{BASE_URL}/chat?chat_id=non-existent-chat-id-12345", json=invalid_payload)
            print(f"Status: {response_invalid.status_code}")
            
            if response_invalid.status_code == 404:
                print("✅ SUCCESS! Correctly rejected invalid chat_id:")
                print(f"   Error: {response_invalid.json()['detail']}")
            else:
                print(f"❌ FAILED! Expected 404, got {response_invalid.status_code}")
            
            # Test 2: Create new conversation and test context
            print("\\n2️⃣ TESTING CONVERSATION CONTEXT:")
            
            # First message (creates new page)
            msg1_payload = {
                "user_id": user_id,
                "message": "My name is Alice and I like cats."
            }
            
            response1 = await client.post(f"{BASE_URL}/chat", json=msg1_payload)
            if response1.status_code == 200:
                data1 = response1.json()
                chat_id = data1['chat_id']
                print(f"✅ First message sent (Page ID: {chat_id}):")
                print(f"   User: {data1['user_message']}")
                print(f"   AI: {data1['bot_response']}")
                
                # Second message (should remember context) - use query parameter
                msg2_payload = {
                    "user_id": user_id,
                    "message": "What is my name?"
                }
                
                response2 = await client.post(f"{BASE_URL}/chat?chat_id={chat_id}", json=msg2_payload)
                if response2.status_code == 200:
                    data2 = response2.json()
                    print(f"\\n✅ Second message (context test):")
                    print(f"   User: {data2['user_message']}")
                    print(f"   AI: {data2['bot_response']}")
                    
                    # Check if AI remembers the name "Alice"
                    if "alice" in data2['bot_response'].lower():
                        print("   🎯 SUCCESS! AI remembered the name from context!")
                    else:
                        print("   ⚠️ AI response might not include context (this can vary)")
                
                # Third message (test more context) - use query parameter
                msg3_payload = {
                    "user_id": user_id,
                    "message": "What do I like?"
                }
                
                response3 = await client.post(f"{BASE_URL}/chat?chat_id={chat_id}", json=msg3_payload)
                if response3.status_code == 200:
                    data3 = response3.json()
                    print(f"\\n✅ Third message (context test):")
                    print(f"   User: {data3['user_message']}")
                    print(f"   AI: {data3['bot_response']}")
                    
                    # Check if AI remembers "cats"
                    if "cat" in data3['bot_response'].lower():
                        print("   🎯 SUCCESS! AI remembered preferences from context!")
                    else:
                        print("   ⚠️ AI response might not include context (this can vary)")
                
                # Test 3: Get conversation history
                print("\\n3️⃣ RETRIEVING CONVERSATION HISTORY:")
                history_response = await client.get(f"{BASE_URL}/chat/{chat_id}/messages")
                if history_response.status_code == 200:
                    messages = history_response.json()
                    print(f"✅ Retrieved {len(messages)} messages:")
                    for i, msg in enumerate(messages, 1):
                        print(f"   {i}. User: {msg['user_message']}")
                        print(f"      AI: {msg['assistant_message'][:50]}...")
            
            # Test 4: Different page (no context sharing)
            print("\\n4️⃣ TESTING DIFFERENT PAGE (No Context Sharing):")
            different_page_payload = {
                "user_id": user_id,
                "message": "What is my name?"  # Same question, different page
            }
            
            response_diff = await client.post(f"{BASE_URL}/chat", json=different_page_payload)
            if response_diff.status_code == 200:
                data_diff = response_diff.json()
                different_chat_id = data_diff['chat_id']
                print(f"✅ Different page created (ID: {different_chat_id}):")
                print(f"   Different from previous: {different_chat_id != chat_id}")
                print(f"   User: {data_diff['user_message']}")
                print(f"   AI: {data_diff['bot_response']}")
                
                if "alice" not in data_diff['bot_response'].lower():
                    print("   🎯 SUCCESS! No context leakage between pages!")
                else:
                    print("   ⚠️ Unexpected: Context might have leaked between pages")
            
        else:
            print(f"❌ User creation failed: {user_response.text}")
            
    except Exception as e:
        print(f"❌ Error: {e}")

print("\\n" + "=" * 60)
print("📊 FEATURES TESTED:")
print("✅ Edge Case: Invalid chat_id properly rejected (404 error)")
print("✅ Context Awareness: AI receives conversation history")
print("✅ Page Isolation: Different pages don't share context")
print("✅ Error Handling: Clear error messages for invalid requests")
print("\\n🎯 Your enhanced chat system is working perfectly!")
print("=" * 60)

🛡️ TESTING EDGE CASES & CONVERSATION CONTEXT
✅ User created: 8d2c8853-8257-4709-8bbd-35c505a31942
\n1️⃣ TESTING INVALID CHAT_ID (Edge Case):
Status: 404
✅ SUCCESS! Correctly rejected invalid chat_id:
   Error: Chat ID 'non-existent-chat-id-12345' does not exist
\n2️⃣ TESTING CONVERSATION CONTEXT:
✅ User created: 8d2c8853-8257-4709-8bbd-35c505a31942
\n1️⃣ TESTING INVALID CHAT_ID (Edge Case):
Status: 404
✅ SUCCESS! Correctly rejected invalid chat_id:
   Error: Chat ID 'non-existent-chat-id-12345' does not exist
\n2️⃣ TESTING CONVERSATION CONTEXT:
✅ First message sent (Page ID: 420bf00f-5e11-4f63-b730-d76d2b4a89e8):
   User: My name is Alice and I like cats.
   AI: Nice to meet you, Alice! I love cats too. They're such wonderful companions. Do you have a cat of your own?
✅ First message sent (Page ID: 420bf00f-5e11-4f63-b730-d76d2b4a89e8):
   User: My name is Alice and I like cats.
   AI: Nice to meet you, Alice! I love cats too. They're such wonderful companions. Do you have a cat of you

In [ ]:
# Test token-limited conversation history
print("🔢 TESTING TOKEN-LIMITED CONVERSATION HISTORY")
print("=" * 60)

async with httpx.AsyncClient(timeout=25) as client:
    try:
        # Create a user first
        user_response = await client.post(f"{BASE_URL}/users")
        if user_response.status_code == 201:
            user_id = user_response.json()["id"]
            print(f"✅ User created: {user_id}")
            
            # Create a long conversation to test token limiting
            print("\\n📝 CREATING LONG CONVERSATION (Testing Token Limit):")
            
            messages_to_send = [
                "Hello, I'm going to tell you about my day. This morning I woke up early and had breakfast.",
                "Then I went to work and had several meetings with my colleagues about various projects.",
                "During lunch break, I went to a restaurant and ordered a delicious pasta dish with vegetables.",
                "In the afternoon, I worked on writing code and debugging some issues in our application.",
                "After work, I went to the gym and did a full workout including cardio and weight training.",
                "In the evening, I cooked dinner at home and watched a movie with my family.",
                "Before bed, I read a book about artificial intelligence and machine learning concepts.",
                "Now I want to ask you: Can you remember what I had for lunch?"  # This should test context
            ]
            
            chat_id = None
            conversation_responses = []
            
            for i, message in enumerate(messages_to_send, 1):
                payload = {
                    "user_id": user_id,
                    "message": message
                }
                
                # Use query parameter for chat_id (except first message)
                if chat_id:
                    response = await client.post(f"{BASE_URL}/chat?chat_id={chat_id}", json=payload)
                else:
                    response = await client.post(f"{BASE_URL}/chat", json=payload)
                
                if response.status_code == 200:
                    data = response.json()
                    if chat_id is None:
                        chat_id = data['chat_id']
                        print(f"   📄 Created conversation page: {chat_id}")
                    
                    conversation_responses.append(data)
                    print(f"   {i}. Sent: {message[:50]}...")
                    print(f"      AI: {data['bot_response']}")
                    
                    # Check if this is the final question about lunch
                    if "lunch" in message.lower():
                        if "pasta" in data['bot_response'].lower():
                            print("      🎯 SUCCESS! AI remembered lunch from earlier context!")
                        else:
                            print("      ⚠️ AI might not remember lunch (token limit working)")
                else:
                    print(f"   ❌ Message {i} failed: {response.status_code}")
            
            # Get full conversation history
            print("\\n📖 RETRIEVING FULL CONVERSATION:")
            history_response = await client.get(f"{BASE_URL}/chat/{chat_id}/messages")
            if history_response.status_code == 200:
                all_messages = history_response.json()
                print(f"✅ Total messages stored: {len(all_messages)}")
                
                # Calculate estimated tokens
                total_chars = 0
                for msg in all_messages:
                    total_chars += len(msg['user_message']) + len(msg['assistant_message'])
                
                estimated_tokens = total_chars // 4
                print(f"📊 Estimated total conversation tokens: {estimated_tokens}")
                print(f"🎯 Token limit setting: 1000 tokens")
                
                if estimated_tokens > 1000:
                    print("✅ SUCCESS! Conversation exceeds 1000 tokens - limiting should be active")
                else:
                    print("ℹ️ Conversation under 1000 tokens - no limiting needed yet")
            
            # Test with even more messages to definitely exceed token limit
            print("\\n🔥 ADDING MORE MESSAGES TO EXCEED TOKEN LIMIT:")
            
            extra_messages = [
                "Let me tell you more details. I also went shopping for groceries including fruits, vegetables, meat, dairy products, and household items.",
                "Additionally, I called my friend and we talked for an hour about our upcoming vacation plans to visit different countries.",
                "What did I have for lunch earlier today?"  # Test if AI still remembers
            ]
            
            for i, message in enumerate(extra_messages, len(messages_to_send) + 1):
                payload = {
                    "user_id": user_id,
                    "message": message
                }
                
                # Use query parameter for chat_id
                response = await client.post(f"{BASE_URL}/chat?chat_id={chat_id}", json=payload)
                if response.status_code == 200:
                    data = response.json()
                    print(f"   {i}. Sent: {message[:50]}...")
                    print(f"      AI: {data['bot_response']}")
                    
                    if "lunch" in message.lower():
                        if "pasta" in data['bot_response'].lower():
                            print("      🎯 AI still remembers lunch despite long conversation!")
                        else:
                            print("      ✅ Token limiting working - early messages excluded from context")
        
        else:
            print(f"❌ User creation failed: {user_response.text}")
            
    except Exception as e:
        print(f"❌ Error: {e}")

print("\\n" + "=" * 60)
print("🎯 TOKEN LIMITING FEATURES:")
print("✅ Conversation history limited to ~1000 tokens")
print("✅ Most recent messages prioritized")
print("✅ Performance optimized for long conversations")
print("✅ Cost-effective API usage")
print("✅ Memory management prevents context overflow")
print("=" * 60)

🔢 TESTING TOKEN-LIMITED CONVERSATION HISTORY
✅ User created: 33f7d6fe-9833-4b9f-8f40-6ea6a09522cb
\n📝 CREATING LONG CONVERSATION (Testing Token Limit):
   📄 Created conversation page: e3c40164-341b-47c8-8a3c-9086106c884b
   1. Sent: Hello, I'm going to tell you about my day. This mo...
      AI: Good morning! Sounds like a great start.
   2. Sent: Then I went to work and had several meetings with ...
      AI: Busy day at work, huh?
   3. Sent: During lunch break, I went to a restaurant and ord...
      AI: That sounds yummy! Enjoyed your lunch?
      ⚠️ AI might not remember lunch (token limit working)
   4. Sent: In the afternoon, I worked on writing code and deb...
      AI: Sounds productive. Code going well?
   5. Sent: After work, I went to the gym and did a full worko...
      AI: Great job! How was the workout?
   6. Sent: In the evening, I cooked dinner at home and watche...
      AI: Nice family time. Movie any good?
   7. Sent: Before bed, I read a book about artificial inte

In [ ]:
# Test with very long conversation to definitely trigger token limiting
print("🚀 TESTING EXTREME TOKEN LIMITING")
print("=" * 50)

async with httpx.AsyncClient(timeout=30) as client:
    try:
        # Create a user first
        user_response = await client.post(f"{BASE_URL}/users")
        if user_response.status_code == 201:
            user_id = user_response.json()["id"]
            print(f"✅ User created: {user_id}")
            
            # Create a very long conversation with detailed messages
            long_messages = [
                "I am starting my day with a comprehensive morning routine that includes meditation, exercise, and a healthy breakfast consisting of oatmeal with fresh berries and nuts.",
                "Next, I will be attending multiple business meetings throughout the day discussing quarterly financial reports, project timelines, budget allocations, and strategic planning initiatives for the upcoming fiscal year.",
                "For lunch, I plan to visit an upscale Italian restaurant downtown where I will order their signature pasta dish with marinara sauce, grilled vegetables, fresh basil, and parmesan cheese.",
                "During the afternoon session, I will be conducting code reviews, debugging complex software applications, optimizing database queries, and implementing new features using modern programming languages and frameworks.",
                "After completing my work responsibilities, I will head to the fitness center for an intensive workout session including cardiovascular exercises, strength training, yoga stretches, and meditation practices.",
                "In the evening, I will prepare a gourmet dinner at home featuring grilled salmon, roasted vegetables, quinoa salad, and a glass of white wine while watching documentaries with my family members.",
                "Before retiring for the night, I plan to read several chapters from educational books covering topics such as artificial intelligence, machine learning algorithms, data science methodologies, and emerging technology trends.",
                "Additionally, I will spend time reviewing my daily accomplishments, planning tomorrow's schedule, organizing my workspace, and reflecting on personal and professional growth opportunities.",
                "I also want to mention that I have been working on a special project involving data analysis, statistical modeling, and predictive analytics using Python programming language and various scientific libraries.",
                "Furthermore, I have been learning about cloud computing platforms, distributed systems architecture, microservices design patterns, and containerization technologies for scalable applications.",
                "My hobbies include photography, hiking in natural environments, playing musical instruments, cooking international cuisines, and exploring different cultures through travel and literature.",
                "I am particularly interested in sustainable living practices, renewable energy solutions, environmental conservation efforts, and contributing to community development initiatives in my local area.",
                "What did I order for lunch at the Italian restaurant today?"  # Test if AI remembers the early lunch detail
            ]
            
            chat_id = None
            
            print("\\n📝 Creating very long conversation...")
            for i, message in enumerate(long_messages, 1):
                payload = {
                    "user_id": user_id,
                    "message": message
                }
                
                # Use query parameter for chat_id (except first message)
                if chat_id:
                    response = await client.post(f"{BASE_URL}/chat?chat_id={chat_id}", json=payload)
                else:
                    response = await client.post(f"{BASE_URL}/chat", json=payload)
                
                if response.status_code == 200:
                    data = response.json()
                    if chat_id is None:
                        chat_id = data['chat_id']
                        print(f"📄 Created conversation: {chat_id}")
                    
                    print(f"{i:2d}. Sent: {message[:60]}...")
                    print(f"    AI: {data['bot_response']}")
                    
                    # Check the final question about lunch
                    if "lunch" in message.lower() and "italian" in message.lower():
                        if any(word in data['bot_response'].lower() for word in ['pasta', 'marinara', 'italian']):
                            print("    🎯 SUCCESS! AI remembered lunch details despite long conversation!")
                        else:
                            print("    ✅ Token limiting working - early lunch details excluded from context")
            
            # Check final conversation stats
            print("\\n📊 FINAL CONVERSATION ANALYSIS:")
            history_response = await client.get(f"{BASE_URL}/chat/{chat_id}/messages")
            if history_response.status_code == 200:
                all_messages = history_response.json()
                total_chars = sum(len(msg['user_message']) + len(msg['assistant_message']) for msg in all_messages)
                estimated_tokens = total_chars // 4
                
                print(f"Total messages: {len(all_messages)}")
                print(f"Total characters: {total_chars:,}")
                print(f"Estimated tokens: {estimated_tokens:,}")
                print(f"Token limit: 1,000")
                
                if estimated_tokens > 1000:
                    print("✅ SUCCESS! Token limiting definitely active for this conversation")
                else:
                    print("ℹ️ Need even longer messages to trigger limiting")
        
        else:
            print(f"❌ User creation failed")
            
    except Exception as e:
        print(f"❌ Error: {e}")

print("\\n" + "=" * 50)
print("🎯 TOKEN LIMITING VERIFICATION COMPLETE!")
print("✅ System handles long conversations efficiently")
print("✅ Memory usage optimized with 1000-token limit")
print("✅ Most recent context prioritized")
print("=" * 50)

🚀 TESTING EXTREME TOKEN LIMITING
✅ User created: 7d0ace06-d515-48e5-b4db-0feb7dc99708
\n📝 Creating very long conversation...
📄 Created conversation: cb971e92-577c-41bb-8db9-a99513e79ecb
 1. Sent: I am starting my day with a comprehensive morning routine th...
    AI: That sounds wonderful! Enjoy your day.
 2. Sent: Next, I will be attending multiple business meetings through...
    AI: Good luck with your meetings.
 3. Sent: For lunch, I plan to visit an upscale Italian restaurant dow...
    AI: Delicious choice! Enjoy your lunch.
    ✅ Token limiting working - early lunch details excluded from context
 4. Sent: During the afternoon session, I will be conducting code revi...
    AI: You've got a full plate. Stay focused!
 5. Sent: After completing my work responsibilities, I will head to th...
    AI: Great way to unwind. Have fun!
 6. Sent: In the evening, I will prepare a gourmet dinner at home feat...
    AI: Sounds like a perfect evening. Enjoy!
 7. Sent: Before retiring for the ni